# Scenario 2 (High Density)

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import product, chain, combinations
from scipy import stats
from IPython.display import display, HTML
%matplotlib inline

def parse_if_number(s):
    try: return float(s)
    except: return True if s=="true" else False if s=="false" else s if s else None

def parse_ndarray(s):
    return np.fromstring(s, sep=' ') if s else None

## Config

In [2]:
inputFile = '2kr-scalars.csv'
repetitionsCount = -1 # -1 = auto-detect
factors = ['R', 'T', 'm', 'D']

# Filter scalars
scalarsFilter = ['Floorplan.userCount', 'Floorplan.coveredUsers:sum', 'Floorplan.collisions:sum', 'Floorplan.activityTime:last']
# Performance indexes
perfIndexes = [
    ('coveredUsersPercent', 'percentage of covered users'),
    ('Floorplan.collisions:sum', 'total number of collisions'),
    ('Floorplan.activityTime:last', 'total broadcast time'),
]

## Load scalars

In [3]:
df = pd.read_csv('exported_data/' + inputFile, converters = {
    'attrvalue': parse_if_number,
    'binedges': parse_ndarray,
    'binvalues': parse_ndarray,
    'vectime': parse_ndarray,
    'vecvalue': parse_ndarray,
})

In [4]:
if repetitionsCount <= 0: # auto-detect
    repetitionsCount = int(df[df.attrname == 'repetition']['attrvalue'].max()) + 1
print('Repetitions:', repetitionsCount)

# Computed
factorsCount = len(factors)
configsCount = 2**factorsCount
totalSims = configsCount*repetitionsCount


display(HTML("<style>div.output_scroll { height: auto; max-height: 48em; }</style>"))
pd.set_option('display.max_rows', totalSims)
pd.set_option('display.max_columns', 2*configsCount*repetitionsCount+1)

Repetitions: 10


In [5]:
scalars = df[(df.type == 'scalar') | ((df.type == 'itervar') & (df.attrname != 'TO')) | ((df.type == 'param') & (df.attrname == 'Floorplan.userCount')) | ((df.type == 'runattr') & (df.attrname == 'repetition'))]
scalars = scalars.assign(qname = scalars.attrname.combine_first(scalars.module + '.' + scalars.name))
scalars.value = scalars.value.combine_first(scalars.attrvalue.astype('float64'))
scalars_wide = scalars.pivot_table(index=['run'], columns='qname', values='value')
scalars_wide.sort_values([*factors, 'repetition'], inplace=True)
count = 0
for index in scalars_wide.index:
    config = count // repetitionsCount
    scalars_wide.loc[index, 'config'] = config
    count += 1
scalars_wide = scalars_wide[['config', 'repetition', *factors, *scalarsFilter]]

In [6]:
# coverage
scalars_wide['coveredUsersPercent'] = scalars_wide['Floorplan.coveredUsers:sum'] / (scalars_wide['Floorplan.userCount'] - 1)

## 2kr

In [7]:
combinedFactors = list(chain(*[list(map(''.join, combinations(''.join(factors), i))) for i in range(1, factorsCount+1)]))
cols = ['exp', 'I'] + combinedFactors
data = [[e, 1] + list(chain(*[list(map(np.prod, combinations(r, i))) for i in range(1, len(r)+1)])) for r,e in zip([list(x) for x in product([-1, 1], repeat=factorsCount)], range(0, configsCount))]
factorsMatrix = pd.DataFrame.from_records(data, columns=cols, index='exp')
for perfIndex, perfIndexDesc in perfIndexes:
    #print("2^k*r analysis for the", perfIndexDesc)
    display(HTML('<h3>2^k*r analysis for the ' + perfIndexDesc + '</h3>'))
    cols = ['exp'] + ['y' + str(i) for i in range(0, repetitionsCount)]
    data = [[i] + scalars_wide[scalars_wide.config == i][perfIndex].values.tolist() for i in range(0, configsCount)]
    ysMatrix = pd.DataFrame.from_records(data, columns=cols, index='exp')
    ysMatrix['mean'] = ysMatrix.mean(axis=1)
    for r in range(0, repetitionsCount):
        ysMatrix['e' + str(r)] = ysMatrix['y' + str(r)] - ysMatrix['mean']

    matrix = pd.concat([factorsMatrix, ysMatrix], axis=1)

    computeCols = ['idx', 'I'] + combinedFactors + ['e' + str(r) for r in range(0, repetitionsCount)]
    data = [['Sum'], ['Mean (Qi)'], ['SSQ']]
    SSE = 0
    for col in computeCols[1:]:
        sumvalue = np.sum(matrix[col].values * matrix['mean'].values)
        data[0].append(sumvalue)
        data[1].append(sumvalue / configsCount)
        if col in ['e' + str(r) for r in range(0, repetitionsCount)]:
            ssq = np.sum(matrix[col].values * matrix[col].values)
            data[2].append(ssq)
            SSE += ssq
        else:
            data[2].append('')
    computeMatrix = pd.DataFrame.from_records(data, columns=computeCols, index='idx')

    data = [['(2^k)*r*(qi)^2'], ['Variation'], ['Variation%']]
    SST = SSE
    for col in combinedFactors:
        qi = computeMatrix.loc['Mean (Qi)', col]
        r2kqi2 = configsCount*repetitionsCount*(qi**2)
        data[0].append(r2kqi2)
        SST += r2kqi2
    data[0].append(SSE)
    idx = 1
    for col in combinedFactors:
        r2kqi2 = data[0][idx]
        var = r2kqi2 / SST
        data[1].append(var)
        data[2].append('{:.2%}'.format(var))
        idx += 1
    data[1].append(SST)
    data[2].append('{:.2%}'.format(SSE/SST))
    resultMatrix = pd.DataFrame.from_records(data, columns=['idx'] + combinedFactors + ['SSE/SST'], index='idx')

    display(matrix)
    display(computeMatrix)
    display(resultMatrix)

    print('SSE =', SSE)
    print('SST =', SST)
    print('Unexplained Variation =', '{:.2%}'.format(SSE/SST))
    print('####################################')
    print()

,I,R,T,m,D,RT,Rm,RD,Tm,TD,mD,RTm,RTD,RmD,TmD,RTmD,y0,y1,y2,y3,y4,y5,y6,y7,y8,y9,mean,e0,e1,e2,e3,e4,e5,e6,e7,e8,e9
exp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1,-1,-1,-1,-1,1,1,1,1,1,1,-1,-1,-1,-1,1,0.877756,0.865731,0.807615,0.789579,0.807615,0.927856,0.907816,0.186373,0.044088,0.869739,0.708417,0.169339,0.157315,0.099198,0.081162,0.099198,0.219439,0.199399,-0.522044,-0.664329,0.161323
1,1,-1,-1,-1,1,1,1,-1,1,-1,-1,-1,1,1,1,-1,0.997996,0.985972,0.953908,0.989980,0.989980,0.927856,0.991984,0.995992,0.997996,1.000000,0.983166,0.014830,0.002806,-0.029259,0.006814,0.006814,-0.055311,0.008818,0.012826,0.014830,0.016834
2,1,-1,-1,1,-1,1,-1,1,-1,1,-1,1,-1,1,1,-1,0.969940,0.983968,0.949900,0.973948,0.949900,0.975952,0.977956,0.973948,0.983968,0.963928,0.970341,-0.000401,0.013627,-0.020441,0.003607,-0.020441,0.005611,0.007615,0.003607,0.013627,-0.006413
3,1,-1,-1,1,1,1,-1,-1,-1,-1,1,1,1,-1,-1,1,1.000000,0.995992,0.987976,0.997996,0.989980,0.993988,0.995992,0.995992,0.995992,1.000000,0.995391,0.004609,0.000601,-0.007415,0.002605,-0.005411,-0.001403,0.000601,0.000601,0.000601,0.004609
4,1,-1,1,-1,-1,-1,1,1,-1,-1,1,1,1,-1,1,-1,0.927856,0.266533,0.857715,0.737475,0.923848,0.861723,0.871743,0.905812,0.861723,0.875752,0.809018,0.118838,-0.542485,0.048697,-0.071543,0.114830,0.052705,0.062725,0.096794,0.052705,0.066733
5,1,-1,1,-1,1,-1,1,-1,-1,1,-1,1,-1,1,-1,1,0.316633,0.987976,0.969940,0.973948,0.917836,0.909820,0.937876,0.841683,0.897796,0.995992,0.874950,-0.558317,0.113026,0.094990,0.098998,0.042886,0.034870,0.062926,-0.033267,0.022846,0.121042
6,1,-1,1,1,-1,-1,-1,1,1,-1,-1,-1,1,1,-1,1,0.993988,0.993988,0.989980,0.995992,0.975952,0.979960,1.000000,0.991984,0.995992,0.991984,0.990982,0.003006,0.003006,-0.001002,0.005010,-0.015030,-0.011022,0.009018,0.001002,0.005010,0.001002
7,1,-1,1,1,1,-1,-1,-1,1,1,1,-1,-1,-1,1,-1,1.000000,1.000000,1.000000,0.997996,1.000000,1.000000,1.000000,1.000000,0.997996,1.000000,0.999599,0.000401,0.000401,0.000401,-0.001603,0.000401,0.000401,0.000401,0.000401,-0.001603,0.000401
8,1,1,-1,-1,-1,-1,-1,-1,1,1,1,1,1,1,-1,-1,0.721443,0.725451,0.833667,0.777555,0.635271,0.781563,0.731463,0.591182,0.669339,0.767535,0.723447,-0.002004,0.002004,0.110220,0.054108,-0.088176,0.058116,0.008016,-0.132265,-0.054108,0.044088


,I,R,T,m,D,RT,Rm,RD,Tm,TD,mD,RTm,RTD,RmD,TmD,RTmD,e0,e1,e2,e3,e4,e5,e6,e7,e8,e9
idx,,,,,,,,,,,,,,,,,,,,,,,,,,
Sum,14.101603,-0.562124,0.264329,0.758918,0.77976,0.22986,-0.402605,0.031062,0.173747,-0.085972,-0.61984,0.108818,0.364529,-0.005812,0.311623,-0.073146,-0.441251,-0.206823,0.219205,0.267345,0.427586,0.250837,0.106806,-0.384140,-0.372848,0.133282
Mean (Qi),0.88135,-0.035133,0.016521,0.047432,0.048735,0.014366,-0.025163,0.001941,0.010859,-0.005373,-0.03874,0.006801,0.022783,-0.000363,0.019476,-0.004572,-0.027578,-0.012926,0.013700,0.016709,0.026724,0.015677,0.006675,-0.024009,-0.023303,0.008330
SSQ,,,,,,,,,,,,,,,,,0.383208,0.361909,0.048232,0.069591,0.076321,0.069104,0.064971,0.363340,0.473667,0.062381


,R,T,m,D,RT,Rm,RD,Tm,TD,mD,RTm,RTD,RmD,TmD,RTmD,SSE/SST
idx,,,,,,,,,,,,,,,,
(2^k)*r*(qi)^2,0.19749,0.043669,0.359973,0.380016,0.033022,0.101307,0.000603,0.018868,0.004619,0.240126,0.007401,0.083051,0.000021,0.060693,0.003344,1.972725
Variation,0.056314,0.012452,0.102646,0.108361,0.009416,0.028888,0.000172,0.00538,0.001317,0.068472,0.00211,0.023682,0.000006,0.017307,0.000954,3.506926
Variation%,5.63%,1.25%,10.26%,10.84%,0.94%,2.89%,0.02%,0.54%,0.13%,6.85%,0.21%,2.37%,0.00%,1.73%,0.10%,56.25%


SSE = 1.9727250091365094
SST = 3.5069264531066144
Unexplained Variation = 56.25%
####################################



,I,R,T,m,D,RT,Rm,RD,Tm,TD,mD,RTm,RTD,RmD,TmD,RTmD,y0,y1,y2,y3,y4,y5,y6,y7,y8,y9,mean,e0,e1,e2,e3,e4,e5,e6,e7,e8,e9
exp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1,-1,-1,-1,-1,1,1,1,1,1,1,-1,-1,-1,-1,1,288.0,244.0,275.0,290.0,353.0,336.0,322.0,62.0,31.0,356.0,255.7,32.3,-11.7,19.3,34.3,97.3,80.3,66.3,-193.7,-224.7,100.3
1,1,-1,-1,-1,1,1,1,-1,1,-1,-1,-1,1,1,1,-1,170.0,168.0,191.0,200.0,163.0,193.0,181.0,180.0,155.0,137.0,173.8,-3.8,-5.8,17.2,26.2,-10.8,19.2,7.2,6.2,-18.8,-36.8
2,1,-1,-1,1,-1,1,-1,1,-1,1,-1,1,-1,1,1,-1,662.0,678.0,675.0,629.0,641.0,706.0,637.0,652.0,634.0,647.0,656.1,5.9,21.9,18.9,-27.1,-15.1,49.9,-19.1,-4.1,-22.1,-9.1
3,1,-1,-1,1,1,1,-1,-1,-1,-1,1,1,1,-1,-1,1,331.0,385.0,394.0,299.0,421.0,296.0,334.0,313.0,338.0,355.0,346.6,-15.6,38.4,47.4,-47.6,74.4,-50.6,-12.6,-33.6,-8.6,8.4
4,1,-1,1,-1,-1,-1,1,1,-1,-1,1,1,1,-1,1,-1,323.0,79.0,290.0,260.0,330.0,383.0,310.0,322.0,259.0,322.0,287.8,35.2,-208.8,2.2,-27.8,42.2,95.2,22.2,34.2,-28.8,34.2
5,1,-1,1,-1,1,-1,1,-1,-1,1,-1,1,-1,1,-1,1,40.0,168.0,149.0,144.0,153.0,128.0,157.0,157.0,131.0,178.0,140.5,-100.5,27.5,8.5,3.5,12.5,-12.5,16.5,16.5,-9.5,37.5
6,1,-1,1,1,-1,-1,-1,1,1,-1,-1,-1,1,1,-1,1,661.0,668.0,624.0,616.0,617.0,629.0,617.0,657.0,618.0,631.0,633.8,27.2,34.2,-9.8,-17.8,-16.8,-4.8,-16.8,23.2,-15.8,-2.8
7,1,-1,1,1,1,-1,-1,-1,1,1,1,-1,-1,-1,1,-1,418.0,438.0,399.0,448.0,443.0,372.0,414.0,440.0,384.0,411.0,416.7,1.3,21.3,-17.7,31.3,26.3,-44.7,-2.7,23.3,-32.7,-5.7
8,1,1,-1,-1,-1,-1,-1,-1,1,1,1,1,1,1,-1,-1,927.0,797.0,661.0,679.0,910.0,474.0,810.0,870.0,756.0,876.0,776.0,151.0,21.0,-115.0,-97.0,134.0,-302.0,34.0,94.0,-20.0,100.0


...

SSE = 2915779.3
SST = 66523287.49375002
Unexplained Variation = 4.38%
####################################



,I,R,T,m,D,RT,Rm,RD,Tm,TD,mD,RTm,RTD,RmD,TmD,RTmD,y0,y1,y2,y3,y4,y5,y6,y7,y8,y9,mean,e0,e1,e2,e3,e4,e5,e6,e7,e8,e9
exp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1,-1,-1,-1,-1,1,1,1,1,1,1,-1,-1,-1,-1,1,33.0,42.0,32.0,26.0,34.0,49.0,39.0,16.0,5.0,40.0,31.6,1.4,10.4,0.4,-5.6,2.4,17.4,7.4,-15.6,-26.6,8.4
1,1,-1,-1,-1,1,1,1,-1,1,-1,-1,-1,1,1,1,-1,51.0,45.0,58.0,60.0,55.0,58.0,60.0,50.0,61.0,46.0,54.4,-3.4,-9.4,3.6,5.6,0.6,3.6,5.6,-4.4,6.6,-8.4
2,1,-1,-1,1,-1,1,-1,1,-1,1,-1,1,-1,1,1,-1,47.0,40.0,27.0,29.0,38.0,44.0,39.0,41.0,37.0,36.0,37.8,9.2,2.2,-10.8,-8.8,0.2,6.2,1.2,3.2,-0.8,-1.8
3,1,-1,-1,1,1,1,-1,-1,-1,-1,1,1,1,-1,-1,1,62.0,43.0,49.0,50.0,47.0,47.0,44.0,46.0,42.0,52.0,48.2,13.8,-5.2,0.8,1.8,-1.2,-1.2,-4.2,-2.2,-6.2,3.8
4,1,-1,1,-1,-1,-1,1,1,-1,-1,1,1,1,-1,1,-1,96.0,60.0,118.0,109.0,135.0,103.0,173.0,92.0,92.0,83.0,106.1,-10.1,-46.1,11.9,2.9,28.9,-3.1,66.9,-14.1,-14.1,-23.1
5,1,-1,1,-1,1,-1,1,-1,-1,1,-1,1,-1,1,-1,1,62.0,183.0,196.0,142.0,134.0,144.0,120.0,148.0,123.0,90.0,134.2,-72.2,48.8,61.8,7.8,-0.2,9.8,-14.2,13.8,-11.2,-44.2
6,1,-1,1,1,-1,-1,-1,1,1,-1,-1,-1,1,1,-1,1,85.0,88.0,72.0,102.0,71.0,84.0,113.0,91.0,96.0,102.0,90.4,-5.4,-2.4,-18.4,11.6,-19.4,-6.4,22.6,0.6,5.6,11.6
7,1,-1,1,1,1,-1,-1,-1,1,1,1,-1,-1,-1,1,-1,105.0,114.0,99.0,108.0,96.0,108.0,115.0,121.0,104.0,84.0,105.4,-0.4,8.6,-6.4,2.6,-9.4,2.6,9.6,15.6,-1.4,-21.4
8,1,1,-1,-1,-1,-1,-1,-1,1,1,1,1,1,1,-1,-1,15.0,10.0,11.0,8.0,6.0,9.0,12.0,11.0,8.0,9.0,9.9,5.1,0.1,1.1,-1.9,-3.9,-0.9,2.1,1.1,-1.9,-0.9


,I,R,T,m,D,RT,Rm,RD,Tm,TD,mD,RTm,RTD,RmD,TmD,RTmD,e0,e1,e2,e3,e4,e5,e6,e7,e8,e9
idx,,,,,,,,,,,,,,,,,,,,,,,,,,
Sum,792.1,-424.1,339.3,-37.1,88.9,-188.9,51.9,-63.7,-48.3,11.7,-45.9,40.7,-8.1,5.1,-13.5,-12.1,-10783.950000,2083.050000,6914.950000,2200.850000,218.750000,2276.550000,8454.550000,1079.150000,-2949.350000,-9494.550000
Mean (Qi),49.50625,-26.50625,21.20625,-2.31875,5.55625,-11.80625,3.24375,-3.98125,-3.01875,0.73125,-2.86875,2.54375,-0.50625,0.31875,-0.84375,-0.75625,-673.996875,130.190625,432.184375,137.553125,13.671875,142.284375,528.409375,67.446875,-184.334375,-593.409375
SSQ,,,,,,,,,,,,,,,,,5731.450000,4976.450000,4574.650000,486.850000,1441.050000,725.450000,5472.450000,998.250000,1603.250000,3324.650000


,R,T,m,D,RT,Rm,RD,Tm,TD,mD,RTm,RTD,RmD,TmD,RTmD,SSE/SST
idx,,,,,,,,,,,,,,,,
(2^k)*r*(qi)^2,112413.00625,71952.80625,860.25625,4939.50625,22302.00625,1683.50625,2536.05625,1458.05625,85.55625,1316.75625,1035.30625,41.00625,16.25625,113.90625,91.50625,29334.5
Variation,0.449329,0.287604,0.003439,0.019744,0.089144,0.006729,0.010137,0.005828,0.000342,0.005263,0.004138,0.000164,0.000065,0.000455,0.000366,250179.99375
Variation%,44.93%,28.76%,0.34%,1.97%,8.91%,0.67%,1.01%,0.58%,0.03%,0.53%,0.41%,0.02%,0.01%,0.05%,0.04%,11.73%


SSE = 29334.5
SST = 250179.99375000008
Unexplained Variation = 11.73%
####################################

